In [1]:
from datetime import datetime
from requests import get
from matplotlib import pyplot as plt
from datetime import datetime
import pandas as pd

In [139]:
#Etherscan API
URL = "https://api.etherscan.io/api"
API_KEY = "NIHBN3PP8W3CHR312I34CM4HK6EZ42NJ3E"

# 1 ETH = GWEI * 10^8
ETH_VALUE = 10 ** 18

In [140]:
# Make a url to access balance of any wallet
def make_api_url(module, action, address, **kwards):

    url = (
        URL
        + f"?module={module}&action={action}&address={Address}&apikey={API_KEY}"
    )

    for key, value in kwards.items():
        url += f"&{key}={value}"
    return url

In [141]:
# Get account balance from api url
def get_account_ballance(address):

    get_balance_url = make_api_url("account", "balance", address, tag="latest", x="2")
    resposne = get(get_balance_url)

    data = resposne.json()
    value = int(data["result"]) / ETH_VALUE
    return value

In [662]:
# example address - this is the top 5 owner address on etherscan 
Address = "0x73bceb1cd57c711feac4224d062b0f6ff338501e"

In [663]:
# Get history of transactions by particular address

get_transaction_url = make_api_url(
    "account",
    "txlist",
    Address,
    startblock=0,
    endblock=99999999,
    page=1,
    offset=10000,
    sort="asc",
)

In [664]:
response = get(get_transaction_url)
data = response.json()["result"]

In [665]:
data

[{'blockNumber': '7181070',
  'timeStamp': '1549418660',
  'hash': '0x5d92f58ef8cb5ebb800e892bfd780ae79e213e12ac2ba8f932534dcdfe3a8ebe',
  'nonce': '188',
  'blockHash': '0x25e078017febb5c5df29139a64f1d7a22ab24bffe1ef31c4d0ee70da31861528',
  'transactionIndex': '172',
  'from': '0xea3ec2a08fee18ff4798c2d4725ded433d94151d',
  'to': '0x73bceb1cd57c711feac4224d062b0f6ff338501e',
  'value': '50000000000000000',
  'gas': '21000',
  'gasPrice': '44000000000',
  'isError': '0',
  'txreceipt_status': '1',
  'input': '0x',
  'contractAddress': '',
  'cumulativeGasUsed': '6006883',
  'gasUsed': '21000',
  'confirmations': '7452459'},
 {'blockNumber': '7204081',
  'timeStamp': '1549833795',
  'hash': '0x097e1a37f05ceb8bd2e94e3997cde25d4fe7019323488ed9436c108058d9f857',
  'nonce': '0',
  'blockHash': '0xb4dc099187a18afd76f25558c36f0f87ff30e224574bc55a344bd81d09f8e28f',
  'transactionIndex': '70',
  'from': '0x73bceb1cd57c711feac4224d062b0f6ff338501e',
  'to': '0xea3ec2a08fee18ff4798c2d4725ded433d9

In [666]:
# We need to add internal transaction to get real balance of account
# Internal transaction is for example when we pay for SmartContracts establishing
get_internal_transaction_url = make_api_url(
    "account",
    "txlistinternal",
    Address,
    startblock=0,
    endblock=99999999,
    page=1,
    offset=10000,
    sort="asc",
)

In [667]:
response2 = get(get_internal_transaction_url)
data2 = response2.json()["result"]

In [668]:
data2

[]

In this example wallet does not make any internal transaction

In [669]:
#creating variable and arrays to append transaction from api
current_balance = 0
balances = []
times = []

In [670]:
# Want to get only few columns
for tx in data:
    to = tx["to"]
    from_adr = tx["from"]
    value = int(tx["value"]) / ETH_VALUE

    if "gasPrice" in tx:
        gas = int(tx["gasUsed"]) * (int(tx["gasPrice"]) / ETH_VALUE)
    else:
        # We need to do that becouse internal transactions do not have gasPrice KEY
        gas = int(tx["gasUsed"]) / ETH_VALUE
    time = datetime.fromtimestamp(int(tx["timeStamp"]))

    # This prints are for showing the history of transaction
    """print("-----------------------")
    print("To", to)
    print("form", from_adr)
    print("Value", value)
    print("gas Cost", gas)
    print("time", time)"""

    money_in = to.lower() == address.lower()

    # We need to devide transaction by income with no gas and outcome with gas
    if money_in:
        current_balance += value
    else:
        current_balance -= value + gas

    balances.append(current_balance)
    times.append(time)

In [673]:
df = pd.DataFrame({"date": times, "balance": balances})
df.to_csv("Data/HistoryOfWallet2.csv", index=False)

After some inwesigation I have seen that the wallet tracker is not perfect tool to track wallets

**WHY?**
1. It can track 1000 days back
2. I can quess that becouse of that some wallets that made big transaction before that period and only sale ETH in this period, the tracker will show negative balance
3. I guess also that some of wallets do short position on ETH and in that case balance could be also negative

**How to improve it?**

I think that the good idea is to reverse the proces:
1. First get the actuall balance
2. Then go back and subtract purchase transactions and add sales transactions